In [1]:
import requests
import time
import re
import openpyxl
import pandas as pd
import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import date

In [2]:
GLOBAL = 'https://www.gipernn.ru'
LINK = 'https://www.gipernn.ru/prodazha-proizvodstvennyh-ploschadey?per-page=50'

Добыча колличества ссылок

In [3]:
html = requests.get(LINK)
soup = BeautifulSoup(html.text, 'html.parser')
count = soup.find('div', class_='count')

count = int(count.text.split(':')[1].strip())
page_count = int(np.ceil(count / 50))

print(count)

35


Составляем массив ссылок

In [4]:
links = []
for page in tqdm(range(page_count + 1), ascii=True):
    html = requests.get(f"{LINK}&page={page}")
    soup = BeautifulSoup(html.text, 'html.parser')
    trs = soup.find('tbody').find_all('tr')
    for tr in trs:
        link = tr.find('a')
        if link:
            link = link.get('href')
            link = GLOBAL + link
            links.append(link)
    links = list(set(links))
print(len(links))   


100%|##########| 2/2 [00:00<00:00,  2.82it/s]

35


Составление своей базы данных

In [5]:
df = pd.DataFrame()
for link in tqdm(links, ascii=True):
    html = requests.get(link)
    soup = BeautifulSoup(html.text, 'html.parser')

    tmp = {} # словарь для временных данных
    tmp['Ссылка'] = link

    header = soup.find('h1')
    if header:
        header = header.text.strip().replace('\xa0', ' ')
    tmp['Заголовок'] = header

    price = soup.find('div', class_ = 'price')
    if price:
        price = price.text.replace('\xa0', ' ').strip().split('  ')
    tmp['Цена'] = price

    description = soup.find('div', class_='formatted-text')
    if description:
        description = description.text.strip().replace('\xa0', ' ')
    tmp['Описание'] = description
    
    try:
        w0 = soup.find('table', id = 'w0')
        if w0:
            w0 = w0.find_all('tr')
            for tr in w0:
                key = tr.find('th').text.strip()
                val = tr.find('td').text.strip()
                tmp[key] = val
    except:
        print('w0 not found')
 
    try:
        w2 = soup.find('table', id = 'w2')
        if w2:
            w2 = w2.find_all('tr')
            for tr in w2:
                key = tr.find('th').text.strip()
                val = tr.find('td').text.strip()
                if key not in tmp.keys(): # записи в 2х таблицах могут 
                    tmp[key] = val # пересекаться, а зачем нам копии?
    except:
        print('w2 not found')

    try:
        properties = soup.find('div', class_ = 'properties m-t-2')
        if properties:
            properties = properties.find_all('div', class_ = 'property')
            for property in properties:
                key = property.find('div', class_ = 'property-name').text.strip()
                val = property.find('div', class_ = 'property-value').text.strip()
                if key not in tmp.keys(): # записи в 2х таблицах могут 
                    tmp[key] = val # пересекаться, а зачем нам копии?
    except:
        print('properties m-t-2 not found')
    
    try:
        buildings = soup.find('div', 'around').find_all('div', 'group')
        for building in buildings:
            name = building.find('div','group-body').find('span').text
            dist = building.find('div','group-body').find('div', 'company').find('span', 'company__distance').text
            tmp[name] = dist   
    except:
        pass

    df = pd.concat([df, pd.DataFrame(tmp)], ignore_index=True)

df['Источник'] = 'gipernn'
df['Сегмент'] = 'Комерческая недвижимость'
df['Подсегмент'] = 'Продажа производственных площадей'
df['Город'] = 'Нижний Новгород'
df['Дата парсинга'] = str(date.today())

100%|##########| 35/35 [00:08<00:00,  4.04it/s]


Посмотрим на результаты вкрадце

In [6]:
df.head()

,Ссылка,Заголовок,Цена,Описание,№,Район,Адрес,Площадь,Этажность,Объект,...,Торг уместен,Электричество,Водопровод,Газ,Канализация,Источник,Сегмент,Подсегмент,Город,Дата парсинга
0,https://www.gipernn.ru/prodazha-zdaniy/ul-mars...,"Продажа здания на ул. Маршала Казакова, д. 3БК...",22 190 000 руб.,Продается производственное здание (в настоящий...,2830993,Канавинский,"ул. Маршала Казакова, д. 3Б",471 м²,2,отдельно стоящее здание,...,NaN,NaN,NaN,NaN,NaN,gipernn,Комерческая недвижимость,Продажа производственных площадей,Нижний Новгород,2022-12-22
1,https://www.gipernn.ru/prodazha-zdaniy/moskovs...,Продажа здания склада пл. 1084 кв. м с ж/д туп...,40 000 000 руб.,None,655559,Московский,NaN,"1 084,3 м²",1,отдельно стоящее здание,...,NaN,NaN,NaN,NaN,NaN,gipernn,Комерческая недвижимость,Продажа производственных площадей,Нижний Новгород,2022-12-22
2,https://www.gipernn.ru/prodazha-imuschestvenny...,Продажа имущественного комплексаАвтозаводский ...,6 500 000 руб.,"Автозаводский район, пос.Доскино.Под строитель...",2856616,Автозаводский,"посёлок Доскино, пос. Доскино",NaN,NaN,имущественный комплекс,...,NaN,NaN,NaN,NaN,NaN,gipernn,Комерческая недвижимость,Продажа производственных площадей,Нижний Новгород,2022-12-22
3,https://www.gipernn.ru/prodazha-zdaniy/ul-kono...,"Продажа здания на ул. Коновалова, д. 5АСормовс...",16 000 000 руб.,"Сормовский р-н, ул. Коновалова,продается база ...",2849805,Сормовский,"ул. Коновалова, д. 5А",600 м²,1,отдельно стоящее здание,...,NaN,NaN,NaN,NaN,NaN,gipernn,Комерческая недвижимость,Продажа производственных площадей,Нижний Новгород,2022-12-22
4,https://www.gipernn.ru/prodazha-pomescheniy/po...,"Продажа помещения на посёлок Стахановский, ул....",11 000 000 руб.,Продается помещение под производство или автор...,2858339,Автозаводский,"посёлок Стахановский, ул. Аркадия Гайдара, д. 93",400 м²,NaN,помещение в нежилом здании,...,NaN,NaN,NaN,NaN,NaN,gipernn,Комерческая недвижимость,Продажа производственных площадей,Нижний Новгород,2022-12-22


Экспортируем таблицу

In [7]:
df.to_excel('manufacture-buy.xlsx', index=False)